# Multi-Modal extraction with Gemini-Flash-1.5 📸 & Langchain ⛓️‍💥
In this notebook we will demo how to use the Gemini-Flash-1.5 model to extract entities from a given text. The model is a multi-modal model that can extract entities from both text and images.

In this example notebook we will touch on the following topics:
1. Extracting image metadata using Langchain and Gemini-Flash-1.5
2. Running the extraction process in parallel across all images in a dataset
3. Add variation to the generated text without using the model temperature parameter

In [ ]:
# Images to extract data from
fruits = ['https://storage.googleapis.com/vectrix-public/fruit/apple.jpeg',
          'https://storage.googleapis.com/vectrix-public/fruit/banana.jpeg',
          'https://storage.googleapis.com/vectrix-public/fruit/kiwi.jpeg',
          'https://storage.googleapis.com/vectrix-public/fruit/peach.jpeg',
          'https://storage.googleapis.com/vectrix-public/fruit/plum.jpeg']

## Passing an image directly to the model
[As described in the LangChain documentation](https://python.langchain.com/v0.2/docs/how_to/multimodal_inputs/), we can use the code below to directly pass an image to the model. This will pass our multi-modal input along with out chat contents to the model.

The code is the same for other LLMs like GPT4o


In [1]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
import base64, httpx

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
image_data = base64.b64encode(httpx.get(fruits[0]).content).decode("utf-8")
message = HumanMessage(
    content=[
        {"type": "text", "text": "describe the fruit in this image"},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ],
)
response = model.invoke([message])
print(response.content)


ModuleNotFoundError: No module named 'langchain_core'

## Extracting structured data from images
The next step is to extract structured data from the image. We can achieve this by combining a [Pydantic parser](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/pydantic/) with a multi-modal message. First we define a Pydantic data model and we will then pass that to the model to extract structured data from the image.

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


# We define a Pydantic model that will be used to parse the output of the model
class Fruit(BaseModel):
    name: str = Field(description="The name of the fruit shown in the image")
    color: str = Field(description="The color of the fruit shown in the image")
    taste: str = Field(description="The taste of the fruit shown in the image")
    marketing_description: str = Field(description="A marketing description of the fruit shown in the image")

parser = PydanticOutputParser(pydantic_object=Fruit)



prompt = ChatPromptTemplate.from_messages([
    (
        "system","Return the requested response object in {language}.\n'{format_instructions}'\n"
    ),
    (
        "human", [
            {
                "type": "image_url",
                "image_url": {"url": "data:image/jpeg;base64,{image_data}"},
            },
        ],
    )
])

chain = prompt | model | parser

# Retrieve the encoded image data
image_data = base64.b64encode(httpx.get(fruits[3]).content).decode("utf-8")


# Run the chain and print the result
print(chain.invoke({"language":"English",
                    "format_instructions":parser.get_format_instructions(),
                    "image_data":image_data}).json(indent=2),)

{
  "name": "Peach",
  "color": "Orange",
  "taste": "Sweet",
  "marketing_description": "A juicy and flavorful peach, perfect for a summer snack or dessert."
}


## Processing all images in parallel and translating the description in two languages
In this example notebook we only have 5 images, but in the case you want to run this on a larger dataset, running this using a regular for loop would be very slow. So, we use Langchains built-in parallel processing capabilities to process all images in parallel.

### Run the requests from above in parallel
By using chain.batch we can now run the extraction process for all the images in parallel. This will be much faster than running the requests sequentially.

Also note that we use the all_images list of dictionaries to feed the chain.batch function. This is because the chain.batch function expects a list of dictionaries as input.

In [ ]:
# Now run this chain in parallel for all the images
all_images = [{"language":"English",
  "format_instructions": parser.get_format_instructions(),
  "image_data":  base64.b64encode(httpx.get(url).content).decode("utf-8")} for url in fruits]

results = chain.batch(all_images, config={"max_concurrency": 5})

for result in results:
    print(result.json(indent=2))

{
  "name": "Apple",
  "color": "Red and Green",
  "taste": "Sweet and Tart",
  "marketing_description": "A crisp and juicy apple with a sweet and tart flavor. Perfect for snacking or baking."
}
{
  "name": "Banana",
  "color": "Yellow",
  "taste": "Sweet",
  "marketing_description": "A delicious and nutritious fruit, perfect for a quick snack or a healthy breakfast. Our bananas are ripe and ready to eat, with a sweet and creamy flavor that everyone will love."
}
{
  "name": "Kiwi",
  "color": "Green",
  "taste": "Sweet and tangy",
  "marketing_description": "The perfect combination of sweet and tangy, our kiwis are bursting with flavor and are a great source of vitamin C. Enjoy them fresh, in smoothies, or as a topping for your favorite desserts."
}
{
  "name": "Apricot",
  "color": "Orange",
  "taste": "Sweet",
  "marketing_description": "A juicy and flavorful apricot, perfect for a summer snack or dessert."
}
{
  "name": "Plum",
  "color": "Red",
  "taste": "Sweet",
  "marketing_des

## Create variations on the output data
As seen in the examples above, all marketing descriptions start with the letter and the last two descriptions start very similar:
> A juicy and flavorful apricot...

> A juicy and flavorful plum...

Output that is very similar is not great for SEO-purposes. For annotating images playing with the temperature of the model isn't the best way to get different results.

#### Instead, we should write a function to forse the model to be more creative. We ask it to start the description with a certain letter and have a description with x amount of words.

In [ ]:
import random

def generate_random_letter():
    letters = ['A', 'B', 'C', 'D', 'M', 'P', 'R', 'S', 'T']
    return str(random.choice(letters))

def generate_random_number():
    return int(random.randint(30, 45))



# A new prompt template that includes the marketing description starting with a letter given in the variable
prompt = ChatPromptTemplate.from_messages([
    (
        "system","Return the requested response object in {language}. Make sure the marketing description starts with the letter '{starting_letter}'\n'{format_instructions}'\n"
    ),
    (
        "human", [
            {
                "type": "image_url",
                "image_url": {"url": "data:image/jpeg;base64,{image_data}"},
            },
        ],
    )
])

# Now run this chain in parallel for all the images
all_images = [{"language":"English",
  "format_instructions": parser.get_format_instructions(),
  "image_data":  base64.b64encode(httpx.get(url).content).decode("utf-8"),
  "starting_letter": generate_random_letter()} for url in fruits] # Make sure you add the starting letter as a variable for the call

chain = prompt | model | parser



results = chain.batch(all_images, config={"max_concurrency": 5})


for result in results:
    print(result.json(indent=2))

{
  "name": "Apple",
  "color": "Red and Green",
  "taste": "Sweet and Tart",
  "marketing_description": "Crisp, juicy, and bursting with flavor, this apple is the perfect snack for any occasion. Enjoy it on its own, or use it in your favorite recipes."
}
{
  "name": "Banana",
  "color": "Yellow",
  "taste": "Sweet",
  "marketing_description": "Bananas are a delicious and versatile fruit that can be enjoyed in many different ways.  They are a good source of potassium and fiber, and they are also a good source of vitamins B6 and C.  Bananas are a great snack, and they can also be used in smoothies, baked goods, and other recipes.  They are also a great source of energy, and they can help to improve your mood."
}
{
  "name": "Kiwi",
  "color": "Green",
  "taste": "Sweet and tangy",
  "marketing_description": "Come and try our delicious kiwi! This green fruit is sweet and tangy, perfect for a healthy snack or a refreshing addition to your smoothies."
}
{
  "name": "Peach",
  "color": "Ora